# GEC Data Science Program
## Level 1, Lab 2

### Imports

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
%matplotlib inline

### Data load

We use Titanic data for this lab again.

In [ ]:
d = pd.read_csv("train.csv", index_col=0)

In [ ]:
d.head()

### More on Visualization

#### A nice visualization library: Seaborn
https://seaborn.pydata.org/

In [ ]:
# install seaborn if needed
!pip install seaborn

In [ ]:
import seaborn as sns

In [ ]:
d.info()

In [ ]:
d1=d[['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']].fillna(0)

In [ ]:
d1.hist(layout=(2,3), figsize=(15,5));

In [ ]:
sns.pairplot(d1, hue="Survived");

In [ ]:
sns.violinplot(x="Pclass", y="Age", hue="Sex", data=d, split=True);

In [ ]:
sns.swarmplot(x="Embarked", y="Fare", hue="Survived", data=d, split=True, color='y');

In [ ]:
sns.heatmap(d1.corr(), annot=True)

In [ ]:
sns.kdeplot(d1.Age, np.log(d1.Fare+1));

In [ ]:
sns.kdeplot(d1.Age[d1.Survived==1], np.log(d1.Fare[d1.Survived==1]+1), shade=True, shade_lowest=False, cmap=plt.cm.Blues);
sns.kdeplot(d1.Age[d1.Survived==0], np.log(d1.Fare[d1.Survived==0]+1), shade=True, shade_lowest=False, cmap=plt.cm.Reds, alpha = 0.7);

## Pre-processing

### Q: What data types do we have in this dataset?

In [ ]:
d.columns

In [ ]:
d.head()

In [ ]:
# 'Survived' : binary
# 'Pclass'   : categorical
# 'Name'     : string
# 'Sex'      : categorical
# 'Age'      : numerical
# 'SibSp'    : numercial
# 'Parch'    : numerical
# 'Ticket'   : string
# 'Fare'     : numerical
# 'Cabin'    : string
# 'Embarked' : categorical

### Q: How many unique values are there in each column?

### Q: Which columns can (or cannot) be used as features?

In [ ]:
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

In [ ]:
target = "Survived"

In [ ]:
df = d[features]

In [ ]:
df.head()

## Encoding

### Q: How do we deal with Categorical features?

In [ ]:
#onehot encoding
pd.get_dummies(df, columns=["Sex"]).head()

In [ ]:
df_encoded = pd.get_dummies(df, columns=["Sex","Pclass","Embarked"], drop_first=True)

In [ ]:
df_encoded.head()

## Missing value imputation

### Q: How many missing values are there? And how do we deal with them?

In [ ]:
df_encoded.isnull().sum()

In [ ]:
df_encoded.Age.fillna(df_encoded.Age.mean(), inplace=True)

In [ ]:
df_encoded.isnull().sum()

Other ways to deal with missing values:
- prediction (e.g. linear regression)
- LVCF (last value carry forward): for time series

## Transformation

### Distribution Normalizing and Scaling

In [ ]:
df_encoded.hist(layout=(2,5), figsize=(15,5));

### Q: Which continous variables are not Normal (Gaussian) distribution. How do we make them more Normal?

In [ ]:
plt.figure(figsize=(10,3))
plt.hist(df_encoded.Fare);

In [ ]:
plt.figure(figsize=(10,3))
plt.hist(np.log(df_encoded.Fare+1));

In [ ]:
dfn = df_encoded.copy()

dfn.Fare = np.log(df_encoded.Fare+1)

dfn.SibSp = np.log(df_encoded.SibSp+1)

dfn.hist(layout=(2,5), figsize=(15,5));

### Q: How do we make all features the same scale (zero mean and one std)?

In [ ]:
dfn.mean()

In [ ]:
dfn.std()

In [ ]:
#Chan Yu:
dfn1 = (dfn-dfn.mean())/dfn.std()

In [ ]:
dfn1.mean()

In [ ]:
dfn1.std()

In [ ]:
dfn1.hist(layout=(2,5), figsize=(15,5));

### Q: How do we scale train and test data?
    a. Scale each dataset independent of the other
    b. First scale all data then split to train and test
    c. Scale training data first, then scale test data the same way (using mean and std of training data)

b and c. Usually c.

### Q: How do we deal with outliers?

### A better way to Standardize the data (using scikit learn)

#### Scikit Learn is a very good MAchine Learning library
http://scikit-learn.org/

In [ ]:
from sklearn import preprocessing

sc = preprocessing.StandardScaler()

dfn = sc.fit_transform(dfn)

Now we can scale the test data the same way. For example:

df_test = sc.transform(df_test)

### Principal Component Analysis (PCA)

In [ ]:
from sklearn import decomposition

pca = decomposition.PCA()

df_pca = pca.fit_transform(dfn)

df_pca

### Q: Plot the first two principal components. Color 'sirvived' and 'died' differently.

In [ ]:
len(dfn1.columns)

In [ ]:
plt.scatter(df_pca[:,0], df_pca[:,1], c=d.Survived)

### Q: How much variation is explained by each Principal Component?

In [ ]:
plt.plot(pca.explained_variance_ratio_)

## Linear Discriminant Analysis (LDA)

In [ ]:
from sklearn import discriminant_analysis

In [ ]:
lda = discriminant_analysis.LinearDiscriminantAnalysis()

In [ ]:
df_lda = lda.fit_transform(dfn, d.Survived)

In [ ]:
plt.hist([df_lda[d.Survived==0],df_lda[d.Survived==1]]);
plt.legend(['died','survived']);
plt.title("LDA");

In [ ]:
plt.hist([df_pca[:,0][d.Survived==0],df_pca[:,0][d.Survived==1]]);
plt.legend(['died','survived']);
plt.title("PCA")

### Q: Which transformation better separates the classes?

LDA

### Q: Include 'title' from HW1 and create some visualizations.

### Q: Is there any way we can extract variables and plot 'Cabin' and 'Ticket' columns?